In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math

!pip install imdbpy
from imdb import IMDb
ia = IMDb()

     |████████████████████████████████| 298 kB 285 kB/s 


2021-11-10 07:46:24,131 INFO [imdbpy.parser.http.piculet] /opt/conda/lib/python3.7/site-packages/imdb/parser/http/piculet.py:262: using lxml


In [2]:
df = pd.read_csv("../input/getting-list-of-k-dramas/kdrama.csv")

In [3]:
df['year'].unique()

array(['2018', '2012', '2014', '2020', '2014–15', '2011', '2005',
       '2016–17', '2013', '2006–07', '2016', '2015', '2007', '2017–18',
       '2013–14', '2020–21', '2010', '2019', '2018–19', '2011–12', '2021',
       '2008', '2002', '2003–04', '2000', '2015–16', '2003', '2010–11',
       '2006', '2004', '2017', '1995', '2009–10', '2005–06', '2004–06',
       '2001', '2019–20', '2009', '2012–13', 'cancelled', '2007–08',
       '1981', '1999', '2008–09', '2017–present', '2016–20', '2004–05',
       '2001–02', '1991–92', '1996–97', '1983', '1999–2000', '1993',
       '1975–78', '2022', '2019–present', '2019–21', '2017–19', '2021–22',
       '2002–04', '2002–03', '1997', '2000–01', '2000–06', '1996',
       '2010–19', '2014-15', '1994–95', '2003–07', '2011–13', '2000–02',
       '1996–98', '1999–2009', '1998–2000', '1994', '2007–2019',
       '2019–2020', '1998', '1997–98'], dtype=object)

In [4]:
df = df[~df['year'].isin(['cancelled', '2022', '2021–22'])]

Next steps:
1. Check every drama, whether it is from SK (to avoid ambiguity due to similar name)
2. List all actors/actresses (maybe limitted to top n)
3. Get IMDB Rating
4. Get other related quantitative parameters


In [5]:
# Filter null-id dramas
df_n = df[df['movie_id'].notnull()]
df_notnull = df_n.copy()

df_n.head()

,title,year,movie_id
0,100 Days My Prince,2018,8199972
2,12 Years Promise,2014,5476252
3,18 Again,2020,12846096
4,365: Repeat the Year,2020,12015466
5,4 Legendary Witches,2014–15,4449190


In [6]:
df_n.shape

(1234, 3)

In [7]:
# Checking
for idx, dr in df_n.iterrows():
    print(idx, dr['title'])
    id_select = 0
    try:
        dr['year'] = int(dr['year'][:4])
        # If multiple IDs
        if isinstance(dr['movie_id'], str) and "," in dr['movie_id']:
            ids = dr['movie_id'].split(',')
            sk_ids = []
            # Check per ID if from SK
            for id in ids:
                drama = ia.get_movie(id)
                if 'countries' in drama.keys():
                    country = ia.get_movie(id)['countries'][0]
                    if country.lower() == 'south korea':
                        sk_ids.append(id)
                else:
                    print(id, "country not found")

            # Get 1st item only
            if len(sk_ids) > 0:
                id_select = int(sk_ids[0])
        else:
            id_select = int(float(dr['movie_id']))

        if not math.isnan(id_select) and id_select != 0:
            d_new = ia.get_movie(id_select)
            df_notnull.loc[idx, 'movie_id'] = id_select

            keys = d_new.keys()

            if 'title' in keys:# Get title
                df_notnull.loc[idx, 'title_new'] = d_new['title']
            else:
                print(d_new['title'], "title not found")

            if 'country codes' in keys: # Get Country
                df_notnull.loc[idx, 'country'] = d_new['country codes']
            else:
                print(d_new['title'], "country code not found")

            if 'cast' in keys: #Get Casts
                casts = d_new['cast']
                cast_IDs = ','.join([x.getID() for x in d_new['cast']])
                df_notnull.loc[idx, 'cast_IDs'] = cast_IDs

                cast_names = ','.join([x['name'] for x in d_new['cast']])
                df_notnull.loc[idx, 'cast_names'] = cast_names
            else:
                print(d_new['title'], "casts not found")

            if 'genres' in keys: # Get genres
                df_notnull.loc[idx, 'genres'] =  ','.join(d_new['genres'])
            else:
                print(d_new['title'], "genres not found")

            if 'rating' in keys: # Get rating
                df_notnull.loc[idx, 'rating'] = d_new['rating']
            else:
                print(d_new['title'], "rating not found")

            if 'year' in keys: # Get Year
                df_notnull.loc[idx, 'year_new'] = int(d_new['year'])
            else:
                print(d_new['title'], "year not found")

            if 'kind' in keys: 
                df_notnull.loc[idx, 'type'] = d_new['kind']
            else:
                print(d_new['title'], "type not found")

        else:
            df_notnull.loc[idx, 'movie_id'] = np.nan
    except:
        print("Error on gathering data")

0 100 Days My Prince


2021-11-10 07:46:26,294 INFO [imdbpy.parser.http.piculet] /opt/conda/lib/python3.7/site-packages/imdb/parser/http/piculet.py:681: using lxml html builder
2021-11-10 07:46:27,491 INFO [imdbpy.parser.http.piculet] /opt/conda/lib/python3.7/site-packages/imdb/parser/http/piculet.py:681: using lxml html builder


2 12 Years Promise


2021-11-10 07:46:29,267 INFO [imdbpy.parser.http.piculet] /opt/conda/lib/python3.7/site-packages/imdb/parser/http/piculet.py:681: using lxml html builder
2021-11-10 07:46:30,406 INFO [imdbpy.parser.http.piculet] /opt/conda/lib/python3.7/site-packages/imdb/parser/http/piculet.py:681: using lxml html builder


3 18 Again


...

2021-11-10 09:37:08,482 INFO [imdbpy.parser.http.piculet] /opt/conda/lib/python3.7/site-packages/imdb/parser/http/piculet.py:681: using lxml html builder
2021-11-10 09:37:10,357 INFO [imdbpy.parser.http.piculet] /opt/conda/lib/python3.7/site-packages/imdb/parser/http/piculet.py:681: using lxml html builder


1403 Yumi's Cells


2021-11-10 09:37:12,930 INFO [imdbpy.parser.http.piculet] /opt/conda/lib/python3.7/site-packages/imdb/parser/http/piculet.py:681: using lxml html builder
2021-11-10 09:37:14,759 INFO [imdbpy.parser.http.piculet] /opt/conda/lib/python3.7/site-packages/imdb/parser/http/piculet.py:681: using lxml html builder


1404 Zombie Detective


2021-11-10 09:37:17,253 INFO [imdbpy.parser.http.piculet] /opt/conda/lib/python3.7/site-packages/imdb/parser/http/piculet.py:681: using lxml html builder
2021-11-10 09:37:19,114 INFO [imdbpy.parser.http.piculet] /opt/conda/lib/python3.7/site-packages/imdb/parser/http/piculet.py:681: using lxml html builder


In [8]:
df_notnull.to_csv('kdrama_v1.csv', index=False)